In [0]:
%run /Repos/a.torralbo@ucl.ac.uk/ccu013_02/atlas/00_Load_helpers

** **outline**: 

1) HES table exploration of admission, and episodes

2) Create pivot of HES_APC with: person_id_deid, admission date, all diagnoses recorded in all the epidsodes, and N days of spell:

 * **ccu013_apc_pivot_admi**

3)  Keep only rows with diagnoses (exclude rows with NULL entries if there were less than 20 diagnoses per episode): 

* **ccu013_apc_pivot_admi_diag**

4)  Create separate table with duration of admission:

* **ccu013_apc_pivot_admi_dur**

5) Join all diagnoses of episodes within the admission with the total  duration of the admissions: 

* **ccu013_apc_pivot_admi_diag_dur**

6) Map to phecodes: 

* **ccu013_phedatav200_admi**

7) Explode: 

* **ccu013_phedatav200_admi_unrolled**

8) Subset cohort and admissions up to 23 Jan 2021:

* **ccu013_phedatav200_admi_unrolled_cohort**

9) Select records related sex for sex relevant phenotypes: 

* **ccu013_phedatav200_admi_unrolled_cohort_sex_dep**  

1) Exploration HES

In [0]:
hes_all = spark.sql(f'''
select * 
from dars_nic_391419_j3w9t_collab.hes_apc_all_years_archive ''')

hes_all.schema

In [0]:
%sql 
select PERSON_ID_DEID, ADMIDATE,DISDATE, SPELBGIN, SPELDUR, SPELEND, EPIKEY, EPISTART, EPIEND, EPIDUR, EPIORDER, DIAG_4_CONCAT 
from dars_nic_391419_j3w9t_collab.hes_apc_all_years_archive  
limit 30;

In [0]:

%sql
select distinct(PERSON_ID_DEID), 
count(distinct(ADMIDATE)), 
count(distinct(EPISTART)), 
count(distinct(EPIKEY)) from dars_nic_391419_j3w9t_collab.hes_apc_all_years_archive where PERSON_ID_DEID = '' group by PERSON_ID_DEID;

In [0]:
%sql select ADMIDATE, DISDATE, SPELDUR, EPIKEY, EPISTART, EPIEND, EPIDUR, DIAG_4_CONCAT 
from  dars_nic_391419_j3w9t_collab.hes_apc_all_years_archive 
where ProductionDate == '2022-01-20 14:58:52.353312' and
PERSON_ID_DEID = '' 
sort by ADMIDATE, EPISTART;

In [0]:
%sql 
SELECT distinct(ADMIDATE), MAX(DISDATE), SPELDUR, EPISTART, MAX(EPIEND), DIAG_4_CONCAT 
FROM dars_nic_391419_j3w9t_collab.hes_apc_all_years_archive
WHERE ProductionDate == '2022-01-20 14:58:52.353312'
AND PERSON_ID_DEID == ''
AND DISDATE IS NOT NULL
AND DISDATE NOT IN('1801-01-01', '1800-01-01')
GROUP BY ADMIDATE, DISDATE, SPELDUR, EPISTART, EPIEND, DIAG_4_CONCAT

In [0]:
%sql 
select distinct(ADMIDATE), DISDATE, SPELDUR, EPISTART, EPIEND, DIAG_4_CONCAT 
from dars_nic_391419_j3w9t_collab.hes_apc_all_years_archive
WHERE ProductionDate = '2022-01-20 14:58:52.353312'
AND PERSON_ID_DEID = '';

get  amidate, spel_dur, all diag


2) Create pivot table of HES APC with patient_id_deid, each admission date and all diagnoses of all episodes wthin the admission, and the total duration of the admission 

In [0]:
import pyspark.sql.functions as f

archive=spark.table("dars_nic_391419_j3w9t_collab.hes_apc_all_years_archive")

diag_fields = [c for c in archive.columns if c.startswith("DIAG_4") and "CONCAT" not in c]

diag_and_n = [(c, int(c.lstrip("DIAG_4"))) for c in diag_fields]

diag_columns = f.array(*(f.struct(f.col(c).alias("value"), f.lit(n).alias("n")) for c, n in diag_and_n))

apc_admistart_pivot = (
    archive
    .where("ProductionDate == '2022-01-20 14:58:52.353312'")
    .select("person_id_deid", 
            f.col("ADMIDATE").alias("date"),
            f.col("SPELDUR").alias("admi_dur"),
            f.explode(diag_columns).alias("diag_4"))
    .selectExpr("person_id_deid", "date", "admi_dur", "diag_4.value as diag_4", "diag_4.n as n") 
)


In [0]:
apc_admistart_pivot.write.mode('overwrite').saveAsTable('dsa_391419_j3w9t_collab.ccu013_apc_pivot_admi')

3) Keep rows with diagnoses (above creates 20 rows as 20 columns for possible diagnoses but not all populated) 
Exclude rows with null diag - hes table episodes up to 20 

In [0]:
phe_admi_diag = spark.sql("""SELECT person_id_deid, date, admi_dur, diag_4 
                             from dsa_391419_j3w9t_collab.ccu013_apc_pivot_admi
                             where diag_4 IS NOT NULL
                             group by person_id_deid, date, admi_dur, diag_4
           """)
phe_admi_diag.createOrReplaceTempView('ccu013_phe_admi_diag')

In [0]:
drop_table('dsa_391419_j3w9t_collab.ccu013_phedatav200_apc_pivot_admi')

In [0]:
phe_admi_diag.write.mode('overwrite').saveAsTable('dsa_391419_j3w9t_collab.ccu013_apc_pivot_admi_diag')

In [0]:
%sql 
select count(*) from dsa_391419_j3w9t_collab.ccu013_apc_pivot_admi_diag;

4) Create separate table with duration of admission (to fix NULL entries if code appears in episode that is not the final one - the only with the duration)

In [0]:
phe_admi_dur = spark.sql("""SELECT person_id_deid, date, max(admi_dur) as admi_dur_new
          FROM dsa_391419_j3w9t_collab.ccu013_apc_admi_diag
          group by person_id_deid, date """)

In [0]:
phe_admi_dur.write.mode('overwrite').saveAsTable('dsa_391419_j3w9t_collab.ccu013_apc_pivot_admi_dur')

5) Join all diag + max duration admission (from final episode) 

In [0]:
phe_admi_dur_diag  = spark.sql("""
                               SELECT 
                               a.person_id_deid, 
                               a.date, 
                               d.admi_dur_new,
                               a.diag_4 
                               from dsa_391419_j3w9t_collab.ccu013_apc_pivot_admi_diag a
                               join dsa_391419_j3w9t_collab.ccu013_apc_pivot_admi_dur d
                               on a.person_id_deid = d.person_id_deid AND a.date = d.date
                               group by a.person_id_deid, a.date, a.diag_4, d.admi_dur_new """)

In [0]:
phe_admi_dur_diag.write.mode('overwrite').saveAsTable('dsa_391419_j3w9t_collab.ccu013_apc_pivot_admi_diag_dur')

6) Map to phecodes:

In [0]:
# Map pivoted HES APC data
# to phecodes and write out
# long file of events.

phedata = spark.sql("""
 SELECT 
  h.person_id_deid, 
  h.date,
  h.admi_dur_new,
  h.DIAG_4 AS source_code,
  h.DIAG_4 AS DIAG_4,
  p.phecode,
  'hes_apc'  AS source
FROM
  dsa_391419_j3w9t_collab.ccu013_phedatav200_apc_pivot_admi_diag_dur h
LEFT JOIN
  dsa_391419_j3w9t_collab.ccu013_lkp_phev200_map p
ON 
  h.diag_4 = p.icd10
WHERE phecode IS NOT NULL
AND YEAR(date) >= 1998
""")

phedata.createOrReplaceTempView('ccu013_phedatav200_admi')
display(phedata)


In [0]:
phedata.count()

In [0]:
phedata.write.mode('overwrite').saveAsTable('dsa_391419_j3w9t_collab.ccu013_phedatav200_admi')

7) Explode: 

In [0]:
phe_unroll = spark.sql("""
SELECT 
    d.person_id_deid,
    d.date,
    d.admi_dur_new as admi_dur,
    d.DIAG_4,
    d.source,
    d.source_code,
    r.phecode_unrolled AS phecode
FROM 
  dsa_391419_j3w9t_collab.ccu013_phedatav200_admi d
LEFT JOIN 
  dsa_391419_j3w9t_collab.ccu013_lkp_phev200_rollmap r
ON 
  d.phecode = r.phecode
AND
 d.DIAG_4 != '-'
 """)

In [0]:
phe_unroll.write.mode('overwrite').saveAsTable('dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_unrolled')

8) Subset for cohort and events aligned with relevant period for this analysis: jan 2021

In [0]:
unrolled_f = spark.sql("""
select 
  p.person_id_deid, 
  p.date as date, 
  p.person_id_deid || '_' || p.date as person_admi_id,
  p.admi_dur,  
  p.source, 
  p.phecode, 
  p.DIAG_4,
  c.sex as person_id_deid_sex
from 
  dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_unrolled p, 
  dars_nic_391419_j3w9t_collab.ccu013_02_dp_skinny_patient_23_01_2020_phe_cohort_joined_1ymort c
where p.person_id_deid = c.NHS_NUMBER_DEID
and c.age_23_01_2020 >= 0 and c.age_23_01_2020 <= 100
and c.sex IN (1, 2)
and c.IMD_quintile not like ('%Unknown')
and p.date <= to_date('2021-01-23', 'yyyy-MM-dd') and p.date >= date(c.dob)
""")

In [0]:
unrolled_f.write.mode('overwrite').saveAsTable('dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_unrolled_cohort')

In [0]:
%sql 
select count(distinct(person_id_deid)) from dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_unrolled_cohort;

In [0]:
%sql 
select max(date) from dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_unrolled_cohort;

In [0]:
%sql
select count(*) from dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_unrolled_cohort;

9) restrict admissions in sex dependent diseases to subset relevant sex

In [0]:
phe_sex = spark.sql("""
select c.person_id_deid, c.date, c.person_admi_id, c.admi_dur, c.source, c.phecode, c.person_id_deid_sex, c.DIAG_4 
from 
  dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_unrolled_cohort c
join 
  dsa_391419_j3w9t_collab.ccu013_lkp_phev200_phedict d
where (c.phecode = d.phecode)
and (d.sex = 0 or d.sex = c.person_id_deid_sex) """)


In [0]:
phe_sex.createOrReplaceTempView('ccu013_phedatav200_admi_unrolled_cohort_sex_dep')

In [0]:
phe_sex.write.mode('overwrite').saveAsTable('dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_unrolled_cohort_sex_dep')

In [0]:
%sql 
select count(distinct(person_id_deid)) from dsa_391419_j3w9t_collab.ccu013_phedatav200_admi_unrolled_cohort_sex_dep ;

test with cohort: 

In [0]:
%sql 
select count(distinct(person_id_deid)) from dsa_391419_j3w9t_collab.ccu013_phedatav200_unrolled_first_sex_dep;